In [1]:
# 크롬 드라이버 업데이트 패키지 설치
!pip install chromedriver_autoinstaller 

In [2]:
from selenium import webdriver
from selenium.webdriver.common import keys
import time

import requests
from bs4 import BeautifulSoup
import json

import numpy as np
import pandas as pd

from datetime import datetime

In [3]:
# 크롬 드라이버 업데이트
import chromedriver_autoinstaller

chromedriver_autoinstaller.install()

'C:\\Users\\yjs01\\anaconda3\\lib\\site-packages\\chromedriver_autoinstaller\\98\\chromedriver.exe'

In [35]:
# statista 사이트 로그인
driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://www.statista.com/companydb/about')

time.sleep(1)
driver.find_element_by_id('onetrust-accept-btn-handler').click()
driver.find_element_by_css_selector('.navMain__item.navMain__item--responsiveEn.navMain__item--layoutDefault').click()

time.sleep(1)
driver.find_element_by_id('loginStat_username').send_keys('********')
driver.find_element_by_id('loginStat_password').send_keys('********')
driver.find_element_by_id('loginStat_submitLogin').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.bGiRCO > a:nth-child(2)"}
  (Session info: chrome=97.0.4692.99)


In [4]:
# # 세션 설정 (whatismybrowser 크롤링이 막힌 관계로 나중에 실행)
# def set_session():
#     driver.execute_script("window.open('https://www.whatismybrowser.com/detect/what-is-my-user-agent', 'new_window')")
#     driver.switch_to.window(driver.window_handles[1])
    
#     user_agent = driver.find_element_by_css_selector('#detected_value > a').text
    
#     s = requests.Session()
#     header = {
#         'User-Agent' : user_agent
#     }
#     s.headers.update(header)
#     for cookie in driver.get_cookies():
#         c = {cookie['name']: cookie['value']}
#         s.cookies.update(c)
#     driver.close()
#     driver.switch_to_window(driver.window_handles[0])
    
#     return s

In [5]:
# 연도별 매출 관련 데이터 추가
def add_rev_data(data, sub_data, key, rnd_digit, unit=''):
    if sub_data[key]:
        oldest_year = min([int(year) for year in sub_data[key].keys()])
        for year in range(oldest_year, 2017):
            if str(year) in sub_data[key].keys():
                sub_data[key].pop(str(year))
    for year in range(2017, 2021):
        if str(year) not in sub_data[key]:
            sub_data[key][str(year)] = None
    sub_data[key] = dict(sorted(sub_data[key].items()))  # 연도별 정렬
    
    for rev in list(sub_data[key].values())[:4]:
        if rev is not None:
            data.extend([str(round(rev, rnd_digit)) + unit])
        else:
            data.extend([np.nan])

In [11]:
def scrap(keyword, start_page, end_page, columns):
    # 세션 설정
#     s = set_session()
#     driver.execute_script("window.open('https://www.whatismybrowser.com/detect/what-is-my-user-agent', 'new_window')")
#     driver.switch_to.window(driver.window_handles[1])
    
#     user_agent = driver.find_element_by_css_selector('#detected_value > a').text
    
    s = requests.Session()
    header = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36'
    }
    s.headers.update(header)
    for cookie in driver.get_cookies():
        c = {cookie['name']: cookie['value']}
        s.cookies.update(c)
#     driver.close()
#     driver.switch_to_window(driver.window_handles[0])

    # 스크래핑 작업
    all_data = list()
    for p in range(start_page, end_page + 1):
        resp = s.get('https://www.statista.com/companydb/search?p=' + str(p) + '&itemsPerPage=100&sortMethod=revenueDesc&currency=USD&q=' + keyword + '&idCountry=0&idBranch=0&idSubBranch=0&typePrivate=1&typePublic=1&statusActive=1&revenueFrom=-1000000000000&revenueTo=1000000000000&employeesFrom=0&employeesTo=100000000')
        soup = BeautifulSoup(resp.text)
        data_items = soup.select_one('section.responsiveSection.margin-bottom-70 > div.responsiveSection__grid > div > div').get('data-var')
        data_items = json.loads(data_items)

        for data in data_items[:10]:
            company_data = list()

            name = data['name']
            location = data['country']
            industry = data['branches'][0]['branch']['desc']
            iso_code = data['currency']['isoCode']
            employees = data['employees']
            ipo_status = 'Public' if data['isStocklisted'] else 'Private'

            revenue = '$'
            if abs(data['revenue']) > 1000:
                revenue += str(round(data['revenue'] / 1000, 1)) + 'b'
            elif abs(data['revenue']) > 1:
                revenue += str(round(data['revenue'], 1)) + 'm'
            elif abs(data['revenue']) > 0.001:
                revenue += str(round(data['revenue'] * 1000, 1)) + 'k'
            elif abs(data['revenue']) > 0:
                revenue += str(round(data['revenue'] * 1000000, 1))
            else:
                revenue = np.nan

            company_data.extend([name, location, industry, revenue, iso_code, employees, ipo_status])

            company_url = 'https://www.statista.com' + data['path']
            c_resp = s.get(company_url)
            c_soup = BeautifulSoup(c_resp.text)

            try:
                sub_data = c_soup.select_one('div#content > div:nth-child(5)').get('data-company-details')
            except AttributeError:
                continue
            sub_data = json.loads(sub_data)

            address = str()
            if sub_data['addressLines']:
                for addr in sub_data['addressLines'].values():
                    address += addr + ', '
                address += sub_data['city']
            phone = sub_data['phone']
            website = sub_data['website']
            email = sub_data['investorRelationsMail']
            est_year = sub_data['foundingYear']
            ceo = sub_data['ceo']
            desc = sub_data['description']

            company_data.extend([address, phone, website, email, est_year, ceo, desc])

            try:
                sub_data_2 = c_soup.select_one('div#content > div:nth-child(5)').get('data-data-points')
            except AttributeError:
                continue
            sub_data_2 = json.loads(sub_data_2)

            add_rev_data(company_data, sub_data_2, 'revenueGrowth', 2, unit='%')
            add_rev_data(company_data, sub_data_2, 'revenuePerEmployee', 4)

            revenue_key = 'revenue' if sub_data_2['revenue'] else 'revenueEstimated'
            if revenue_key == 'revenueEstimated' or max(sub_data_2['revenue'].values()) >= 10**8:
                for year in sub_data_2[revenue_key].keys():
                    sub_data_2[revenue_key][year] /= 1000000
            add_rev_data(company_data, sub_data_2, revenue_key, 1)

            all_data.append(company_data)
            
            time.sleep(30)

    all_columns = [
        'Name', 'Country', 'Industry', 'Revenue', 'ISO Code', 'Employees', 'IPO Status', 
        'Address', 'Tel', 'Homepage', 'Email', 'Founding year', 'CEO', 'Highlights'
    ]
    years = range(2017, 2021)
    all_columns.extend(['Revenue YoY Growth ' + str(year) for year in years])
    all_columns.extend(['Revenue per Employee ' + str(year) for year in years])
    all_columns.extend(['Revenue ' + str(year) for year in years])
    all_data = pd.DataFrame(all_data, columns=all_columns)[columns]

    now = datetime.today().strftime('%y%m%d_%H%M%S')
    all_data.to_csv('data/data_' + now + '.csv', encoding='utf-8-sig')

In [ ]:
from flask import Flask, render_template, request, redirect

app = Flask('__name__')


@app.route('/')
def main():
    return render_template('main.html')


@app.route('/', methods=['GET', 'POST'])
def execute_scraping():
    if request.method == 'POST':
        keyword = request.form['keyword']
        start_page = int(request.form['start-page'])
        end_page = int(request.form['end-page'])
        columns = request.form.getlist('column')
        
        scrap(keyword, start_page, end_page, columns)

        return redirect('/')


if __name__ == '__main__':
    app.run(host='127.0.0.1', port=8081)

 * Serving Flask app "__name__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:8081/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [26/Jan/2022 15:29:41] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Jan/2022 15:29:41] "GET /static/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Jan/2022 15:29:42] "GET /favicon.ico HTTP/1.1" 404 -
